In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
from time import sleep

In [2]:
# 지번주소로 도로명주소와 좌표를 다운로드함. 출처 - 행정안전부 도로명주소 안내시스템 개발자센터

def fetch_coor_data(df):
    df = df.copy()
    
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDUwNDE1NTMwMDExMTEyNzc='
    coor_base_url = 'https://www.juso.go.kr/addrlink/addrCoordApi.do?'
    coor_apikey = 'devU01TX0FVVEgyMDIxMDIyNjExMDQwMDExMDg1ODU='
    # 도로명주소 검색 api key 만료일 2021년 8월 2일
    # 좌표검색 api key 만료일 2021년 5월 27일
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    coor_data_df = pd.DataFrame({'지번주소':[], '도로명주소':[], '역명':[], '노선명':[], 'x좌표':[], 'y좌표':[]})
    
    for i in tqdm(range(df.shape[0])):
        address_no_zero = df['주소'].loc[i]
        station_name = df['역명'].loc[i]
        line_name = df['노선명'].loc[i]
        
        if station_name.endswith('역') == False:
            station_name += '역'
            
        roadname_result = None
        coor_result = None
        coor_info = None
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=5' + '&keyword=' \
                                       + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        sleep(1)
        roadname_info = roadname_result.json()['results']['juso']
        
        the_idx = None        
        if roadname_info == []:
            roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=5' + '&keyword=' \
                                       + station_name + '&resultType=json' + '&hstryYn=Y')
            sleep(1)
            roadname_info = roadname_result.json()['results']['juso']
            
        for j in range(len(roadname_info)):
            if (roadname_info[j]['bdNm'] == station_name)|((station_name in roadname_info[j]['bdNm'])&(line_name in roadname_info[j]['bdNm'])):
                the_idx = j
                break
        
        if the_idx == None:
            roadname_info = roadname_info[0]
        else:
            roadname_info = roadname_info[the_idx]
        
        coor_result = requests.get(coor_base_url + 'confmKey=' + coor_apikey + '&admCd=' + roadname_info['admCd']\
                                   + '&rnMgtSn=' + roadname_info['rnMgtSn'] + '&udrtYn=' + roadname_info['udrtYn']\
                                   + '&buldMnnm=' + roadname_info['buldMnnm'] + '&buldSlno=' + roadname_info['buldSlno']\
                                   + '&resultType=json')
        sleep(1)
        #return coor_result
        
        if coor_result.json()['results']['juso'] == None:
            print(address_no_zero)
            continue
        
        coor_info = coor_result.json()['results']['juso'][0]
        
        coor_x = pd.to_numeric(coor_info['entX'])
        coor_y = pd.to_numeric(coor_info['entY'])
        
        roadname_addr = roadname_info['roadAddrPart1']
        
        coor_data_df.loc[i,'지번주소'] = address_no_zero
        coor_data_df.loc[i,'도로명주소'] = roadname_addr
        coor_data_df.loc[i, '역명'] = station_name
        coor_data_df.loc[i, '노선명'] = line_name
        coor_data_df.loc[i, 'x좌표'] = coor_x
        coor_data_df.loc[i, 'y좌표'] = coor_y
        
    
    
    return coor_data_df

In [3]:
basedir = './지하철역/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.xlsx')]
filenames

['서울교통공사_역주소 및 전화번호_20200715.xlsx', '지하철역.xlsx', '지하철역_역명_주소.xlsx']

In [4]:
subway_df = pd.read_excel(basedir + filenames[-1]).dropna().reset_index(drop=True)
print(subway_df.shape)
subway_df.head()

(743, 3)


,역명,노선명,주소
0,인천,1,인천광역시 중구 제물량로 269
1,동인천,1,인천광역시 중구 참외전로 121
2,도원,1,인천광역시 동구 참외전로 245
3,제물포,1,인천광역시 미추홀구 경인로 129
4,도화,1,인천광역시 미추홀구 숙골로 24번길 9


In [5]:
def rename_station_name(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if x.endswith('역'):
            return x
        else:
            return x + '역'

In [6]:
subway_df['역명'] = subway_df['역명'].apply(rename_station_name)

In [7]:
def rename_line_name(x):
    # x is a number or a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        for i in range(10):
            if x.endswith(str(i)):
                return x + '호선'
        return x + '선'

In [8]:
subway_df.head()

,역명,노선명,주소
0,인천역,1,인천광역시 중구 제물량로 269
1,동인천역,1,인천광역시 중구 참외전로 121
2,도원역,1,인천광역시 동구 참외전로 245
3,제물포역,1,인천광역시 미추홀구 경인로 129
4,도화역,1,인천광역시 미추홀구 숙골로 24번길 9


In [10]:
subway_df['노선명'] = subway_df['노선명'].apply(str)

In [11]:
subway_df['노선명'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '인천1', '인천2', '수인분당',
       '신분당', '경의중앙', '공항', '경춘', '의정부경전철', '용인경전철', '경강', '우이신설경전철',
       '서해', '김포'], dtype=object)

In [12]:
ui_df = subway_df[subway_df['노선명'].str.contains('우이신설')]

In [13]:
subway_df.loc[ui_df.index, '노선명'] = '우이'

In [14]:
uijungbu_df = subway_df[subway_df['노선명'].str.contains('의정부경')]

In [15]:
subway_df.loc[uijungbu_df.index, '노선명'] = '의정부경'

In [16]:
yongin_df = subway_df[subway_df['노선명'].str.contains('용인경')]

In [17]:
subway_df.loc[yongin_df.index, '노선명'] = '용인경'

In [18]:
subway_df['노선명'].unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '인천1', '인천2', '수인분당',
       '신분당', '경의중앙', '공항', '경춘', '의정부경', '용인경', '경강', '우이', '서해', '김포'],
      dtype=object)

In [20]:
seoul_df = subway_df[subway_df['주소'].str.contains('서울특별시')]
print(seoul_df.shape)
seoul_df.head()

(379, 3)


,역명,노선명,주소
16,온수역,1,서울특별시 구로구 부일로 872
17,오류동역,1,서울특별시 구로구 경인로 20길 13
18,개봉역,1,서울특별시 구로구 경인로 40길 47
19,구일역,1,서울특별시 구로구 구일로 133
20,구로역,1,서울특별시 구로구 구로중앙로 174


In [21]:
subway_coor_df = fetch_coor_data(subway_df)

100%|████████████████████████████████████████████████████████████████████████████████| 743/743 [29:22<00:00,  2.37s/it]


In [22]:
print(subway_coor_df.shape)
subway_coor_df.head()

(743, 6)


,지번주소,도로명주소,역명,노선명,x좌표,y좌표
0,인천광역시 중구 제물량로 269,인천광역시 중구 제물량로 269,인천역,1,921930.430042,1.942218e+06
1,인천광역시 중구 참외전로 121,인천광역시 중구 참외전로 121,동인천역,1,923228.488111,1.942197e+06
2,인천광역시 동구 참외전로 245,인천광역시 동구 참외전로 245,도원역,1,924164.778287,1.941409e+06
3,인천광역시 미추홀구 경인로 129,인천광역시 미추홀구 경인로 129,제물포역,1,925441.957270,1.941177e+06
4,인천광역시 미추홀구 숙골로 24번길 9,인천광역시 미추홀구 숙골로24번길 9,도화역,1,926458.789932,1.941091e+06


In [23]:
for v in subway_coor_df['역명'].unique():
    v_df = subway_coor_df[subway_coor_df['역명'] == v]
    
    if v_df.shape[0] > 1:
        for i in range(v_df.shape[0]):
            v_df.iloc[i, 2] = v + '_{}'.format(i)
        
        subway_coor_df.loc[v_df.index, '역명'] = v_df['역명']

In [24]:
subway_coor_df['역명'].nunique()

743

In [25]:
subway_coor_df['역명'].unique()

array(['인천역_0', '동인천역', '도원역', '제물포역', '도화역', '주안역_0', '간석역', '동암역',
       '백운역', '부평역_0', '부개역', '송내역', '중동역', '부천역', '소사역_0', '역곡역',
       '온수역_0', '오류동역', '개봉역', '구일역', '구로역', '신도림역_0', '영등포역', '신길역_0',
       '대방역', '노량진역_0', '용산역_0', '남영역', '서울역_0', '시청역_0', '종각역',
       '종로3가역_0', '종로5가역', '동대문역_0', '동묘앞역_0', '신설동역_0', '제기동역', '청량리역_0',
       '회기역_0', '외대앞역', '신이문역', '석계역_0', '광운대역_0', '월계역', '녹천역', '창동역_0',
       '방학역', '도봉역', '도봉산역_0', '망월사역', '회룡역_0', '의정부역', '가능역', '녹양역',
       '양주역', '덕계역', '덕정역', '지행역', '동두천중앙역', '보산역', '동두천역', '소요산역',
       '가산 디지털단지역', '독산역', '금천구청역', '광명역', '석수역', '관악역', '안양역', '명학역',
       '금정역_0', '군포역', '당정역', '의왕역', '성균관대역', '화서역', '수원역_0', '세류역',
       '병점역', '서동탄역', '세마역', '오산대역', '오산역', '진위역', '송탄역', '서정리역', '평택지제역',
       '평택역', '성환역', '직산역', '두정역', '천안역', '봉명역', '쌍용역', '아산역', '배방역',
       '온양온천역', '신창역', '까치산역_0', '신정네거리역', '양천구청역', '도림천역', '신도림역_1',
       '문래역', '영등포구청역_0', '당산역_0', '합정역_0', '홍대입구역_0', '신촌역_0', '이대역',
       '아현역', 

In [26]:
subway_coor_df.to_csv('./지하철역/subway_coor_2.csv', index=False)